In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/sample.csv.csv
/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv
/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv')
train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-06-28 17:20:21,2023-06-28 16:34:45,1.0,2.14,1.0,N,120,9,Credit Card,2.5,7.165589,0.0,1.0,20.64,2.5,0.0
1,0,2023-06-29 23:05:01,2023-06-29 22:01:35,1.0,2.70,1.0,N,15,215,Credit Card,3.5,6.067401,0.0,1.0,25.55,2.5,0.0
2,1,2023-06-30 10:19:31,2023-06-30 11:13:10,1.0,1.15,1.0,N,167,223,Credit Card,0.0,4.111547,0.0,1.0,17.64,2.5,0.0
3,0,2023-06-29 13:23:09,2023-06-29 14:20:01,1.0,0.40,1.0,N,128,239,Credit Card,2.5,6.411079,0.0,1.0,12.80,2.5,0.0
4,1,2023-06-29 22:03:32,2023-06-29 22:22:22,3.0,1.10,1.0,N,203,52,Credit Card,1.0,4.769377,0.0,1.0,18.00,2.5,0.0


In [3]:
train.shape

(175000, 17)

In [4]:
sample_data = pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/sample.csv.csv')
sample_data.head()

,ID,total_amount
0,1,24.456348
1,2,24.374058
2,3,19.878154
3,4,25.015569
4,5,22.252489


In [5]:
test = pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv')
test.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee
0,1,2023-06-29 00:21:20,2023-06-29 00:25:20,1.0,4.95,1.0,N,20,3,Credit Card,1.0,6.067612,0.0,1.0,2.5,0.0
1,1,2023-06-30 17:44:43,2023-06-30 17:53:13,1.0,2.10,1.0,N,9,81,Credit Card,2.5,6.191269,0.0,1.0,2.5,0.0
2,1,2023-06-29 18:17:04,2023-06-29 19:23:48,1.0,0.95,1.0,N,92,90,Cash,2.5,3.983872,0.0,1.0,2.5,0.0
3,0,2023-06-30 21:33:53,2023-06-30 21:46:20,1.0,0.80,1.0,N,19,102,Credit Card,3.5,6.839341,0.0,1.0,2.5,0.0
4,1,2023-06-29 14:53:54,2023-06-29 15:22:17,1.0,4.01,1.0,N,131,229,Cash,0.0,1.468943,0.0,1.0,0.0,0.0


In [6]:
train.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type              object
extra                    float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
Airport_fee              float64
dtype: object

Dividing features and Labels:

In [7]:
X = train.drop(columns=['total_amount'])
y = train['total_amount']

Converting Datetime columns to useful data:

In [8]:
X['travel_time'] = ((pd.to_datetime(X['tpep_dropoff_datetime'])- pd.to_datetime(X['tpep_pickup_datetime'])).dt.seconds)/60
X = X.drop(columns = ['tpep_dropoff_datetime','tpep_pickup_datetime'])

In [9]:
test['travel_time'] = ((pd.to_datetime(test['tpep_dropoff_datetime'])- pd.to_datetime(test['tpep_pickup_datetime'])).dt.seconds)/60
test = test.drop(columns = ['tpep_dropoff_datetime','tpep_pickup_datetime'])

Handling Null Values:

In [10]:
X.isnull().sum()

VendorID                    0
passenger_count          6077
trip_distance               0
RatecodeID               6077
store_and_fwd_flag       6077
PULocationID                0
DOLocationID                0
payment_type                0
extra                       0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
congestion_surcharge     6077
Airport_fee              6077
travel_time                 0
dtype: int64

In [11]:
print(X['passenger_count'].value_counts())
print(X['RatecodeID'].value_counts())
print(X['store_and_fwd_flag'].value_counts())
print(X['congestion_surcharge'].value_counts())
print(X['Airport_fee'].value_counts())
print(X['VendorID'].value_counts())

passenger_count
1.0    128534
2.0     24316
3.0      6018
4.0      3668
0.0      2818
5.0      1970
6.0      1596
8.0         2
9.0         1
Name: count, dtype: int64
RatecodeID
1.0     158652
2.0       7314
5.0       1036
99.0       748
3.0        727
4.0        446
Name: count, dtype: int64
store_and_fwd_flag
N    167729
Y      1194
Name: count, dtype: int64
congestion_surcharge
 2.5    153212
 0.0     14325
-2.5      1386
Name: count, dtype: int64
Airport_fee
 0.00    153074
 1.75     15590
-1.75       259
Name: count, dtype: int64
VendorID
1    127340
0     47597
2        63
Name: count, dtype: int64


Replace missing or unknown values with the most frequent category for categorical values:

In [12]:
freq1 = X['store_and_fwd_flag'].value_counts()
most_freq1 = freq1[freq1 == freq1.max()].index
max_val1 = most_freq1[0]
X['store_and_fwd_flag'].fillna(max_val1, inplace=True)

freq2 = test['store_and_fwd_flag'].value_counts()
most_freq2 = freq2[freq2 == freq2.max()].index
max_val2 = most_freq2[0]
test['store_and_fwd_flag'].fillna(max_val2, inplace=True)

In [13]:
X['payment_type'].value_counts()

payment_type
Credit Card    135257
Cash            30141
Wallet           6077
unknown          2333
UPI              1192
Name: count, dtype: int64

Replacing with 'unknown' values with the most frequent values:

In [14]:
freq3 = X['payment_type'].value_counts()
most_freq3 = freq3[freq3 == freq3.max()].index
max_val3 = most_freq3[0]
X['payment_type'].replace('unknown', max_val3, inplace = True)

freq4 = test['payment_type'].value_counts()
most_freq4 = freq4[freq4 == freq4.max()].index
max_val4 = most_freq4[0]
test['payment_type'].replace('unknown', max_val4, inplace = True)

Imputing null values with mean of the numerical columns:

In [15]:
from sklearn.impute import SimpleImputer
cols = ['passenger_count', 'trip_distance', 'RatecodeID', 'extra', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'Airport_fee']
imputer = SimpleImputer(strategy='mean')
X[cols] = imputer.fit_transform(X[cols])

In [16]:
test_cols = ['passenger_count', 'trip_distance', 'RatecodeID', 'extra', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'Airport_fee']
test[test_cols] = imputer.fit_transform(test[test_cols])

In [17]:
#df['passenger_count'] = df['passenger_count'].fillna(df['passenger_count'].mean())
#df['RatecodeID'] = df['RatecodeID'].fillna(df['RatecodeID'].mean())
#df['store_and_fwd_flag'] = df['store_and_fwd_flag'].fillna(df['store_and_fwd_flag'].mode()[0])
#df['congestion_surcharge'] = df['congestion_surcharge'].fillna(df['congestion_surcharge'].mean())
#df['Airport_fee'] = df['Airport_fee'].fillna(df['Airport_fee'].mean())

In [18]:
X.isna().sum()

VendorID                 0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
extra                    0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
congestion_surcharge     0
Airport_fee              0
travel_time              0
dtype: int64

In [19]:
test.isna().sum()

VendorID                 0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
extra                    0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
congestion_surcharge     0
Airport_fee              0
travel_time              0
dtype: int64

Encoding categorical columns using One-hot Encoding:

In [20]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

encoded = encoder.fit_transform(X[['store_and_fwd_flag']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['store_and_fwd_flag']))
X = pd.concat([X, encoded_df], axis=1)

In [21]:
encoded = encoder.fit_transform(test[['store_and_fwd_flag']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['store_and_fwd_flag']))
test = pd.concat([test, encoded_df], axis=1)

In [22]:
encoded = encoder.fit_transform(X[['payment_type']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['payment_type']))
X = pd.concat([X, encoded_df], axis=1)

encoded = encoder.fit_transform(test[['payment_type']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['payment_type']))
test = pd.concat([test, encoded_df], axis=1)

In [23]:
encoded = encoder.fit_transform(X[['VendorID']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['VendorID']))
X = pd.concat([X, encoded_df], axis=1)

encoded = encoder.fit_transform(test[['VendorID']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['VendorID']))
test = pd.concat([test, encoded_df], axis=1)

In [24]:
encoded = encoder.fit_transform(X[['RatecodeID']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['RatecodeID']))
X = pd.concat([X, encoded_df], axis=1)

encoded = encoder.fit_transform(test[['RatecodeID']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['RatecodeID']))
test = pd.concat([test, encoded_df], axis=1)

In [25]:
X = X.drop(columns=['payment_type','store_and_fwd_flag','VendorID','RatecodeID'])
test = test.drop(columns=['payment_type','store_and_fwd_flag','VendorID','RatecodeID'])

In [26]:
#Matching the columns at Test and Train Datasets after One-Hot Oncoding:

X['RatecodeID_1.5670143713319924'] = 0
X['RatecodeID_6.0'] = 0
test['RatecodeID_1.51830715769907'] = 0

In [27]:
# Get the column order from the training dataset
column_order = X.columns
# Reorder the columns in the test dataset
test = test[column_order]

In [28]:
X.head()

,passenger_count,trip_distance,PULocationID,DOLocationID,extra,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,...,VendorID_2,RatecodeID_1.0,RatecodeID_1.51830715769907,RatecodeID_2.0,RatecodeID_3.0,RatecodeID_4.0,RatecodeID_5.0,RatecodeID_99.0,RatecodeID_1.5670143713319924,RatecodeID_6.0
0,1.0,2.14,120,9,2.5,7.165589,0.0,1.0,2.5,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,1.0,2.70,15,215,3.5,6.067401,0.0,1.0,2.5,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,1.0,1.15,167,223,0.0,4.111547,0.0,1.0,2.5,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,1.0,0.40,128,239,2.5,6.411079,0.0,1.0,2.5,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,3.0,1.10,203,52,1.0,4.769377,0.0,1.0,2.5,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


Potential Outlier Detection for numeric values:

In [29]:
#X.head()

In [30]:
'''import matplotlib.pyplot as plt
numeric_features = ['passenger_count', 'trip_distance', 'travel_time', 
                   'tip_amount','improvement_surcharge','congestion_surcharge','Airport_fee']
for feature in numeric_features:
    plt.figure(figsize=(6, 4))
    plt.scatter(X[feature], X[feature], label=feature)
    plt.title(f'Scatter Plot of {feature}')
    plt.xlabel(feature)
    plt.ylabel(feature)
    plt.legend()
    plt.show()'''

"import matplotlib.pyplot as plt\nnumeric_features = ['passenger_count', 'trip_distance', 'travel_time', \n                   'tip_amount','improvement_surcharge','congestion_surcharge','Airport_fee']\nfor feature in numeric_features:\n    plt.figure(figsize=(6, 4))\n    plt.scatter(X[feature], X[feature], label=feature)\n    plt.title(f'Scatter Plot of {feature}')\n    plt.xlabel(feature)\n    plt.ylabel(feature)\n    plt.legend()\n    plt.show()"

In [31]:
'''# IQR Outlier removal method:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_cleaned = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_cleaned


X = remove_outliers_iqr(X, 'tip_amount')
X = remove_outliers_iqr(X, 'trip_distance')'''

"# IQR Outlier removal method:\ndef remove_outliers_iqr(df, column):\n    Q1 = df[column].quantile(0.25)\n    Q3 = df[column].quantile(0.75)\n    IQR = Q3 - Q1\n    lower_bound = Q1 - 1.5 * IQR\n    upper_bound = Q3 + 1.5 * IQR\n    df_cleaned = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]\n    return df_cleaned\n\n\nX = remove_outliers_iqr(X, 'tip_amount')\nX = remove_outliers_iqr(X, 'trip_distance')"

In [32]:
'''numeric_features = ['passenger_count', 'trip_distance', 'travel_time', 
                   'tip_amount','improvement_surcharge','congestion_surcharge','Airport_fee']
for feature in numeric_features:
    plt.figure(figsize=(6, 4))
    plt.scatter(X[feature], X[feature], label=feature)
    plt.title(f'Scatter Plot of {feature}')
    plt.xlabel(feature)
    plt.ylabel(feature)
    plt.legend()
    plt.show()'''

"numeric_features = ['passenger_count', 'trip_distance', 'travel_time', \n                   'tip_amount','improvement_surcharge','congestion_surcharge','Airport_fee']\nfor feature in numeric_features:\n    plt.figure(figsize=(6, 4))\n    plt.scatter(X[feature], X[feature], label=feature)\n    plt.title(f'Scatter Plot of {feature}')\n    plt.xlabel(feature)\n    plt.ylabel(feature)\n    plt.legend()\n    plt.show()"

In [33]:
#X

In [34]:
'''temp = X[X['tip_amount'] > 150]
temp['tip_amount'].value_counts()'''

"temp = X[X['tip_amount'] > 150]\ntemp['tip_amount'].value_counts()"

In [35]:
'''#Remove outlier:
X = X[X['tip_amount'] <= 150]'''

"#Remove outlier:\nX = X[X['tip_amount'] <= 150]"

In [36]:
'''temp = X[X['trip_distance'] > 100]
temp['trip_distance'].value_counts()'''

"temp = X[X['trip_distance'] > 100]\ntemp['trip_distance'].value_counts()"

Test-Train Split

In [37]:
#Necessary Common Libraries:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer, r2_score
from sklearn.model_selection import cross_val_score

In [38]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)

In [39]:
#from sklearn.model_selection import train_test_split
#X = df.drop(columns=['payment_type','store_and_fwd_flag','tpep_pickup_datetime', 'tpep_dropoff_datetime','total_amount'])
#y = df['total_amount']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Base Model:

In [40]:
from sklearn.dummy import DummyRegressor

#scaler = MinMaxScaler()
scaler = StandardScaler()

dummy_regressor = DummyRegressor(strategy='mean')

dummy_pipeline = Pipeline([('scaler', scaler),('regressor', dummy_regressor)])
dummy_pipeline.fit(X_train, y_train)

y_pred1 = dummy_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred1)
mse = mean_squared_error(y_val, y_pred1)
rmse = mean_squared_error(y_val, y_pred1, squared=False)
r2 = r2_score(y_val, y_pred1)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")


Mean Absolute Error: 16.42
Mean Squared Error: 656.83
Root Mean Squared Error: 25.63
R-squared (R2) Score: -0.00


In [41]:
r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(dummy_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())

Cross-Validation R-squared Scores: [-1.73444158e-06 -1.13478103e-04 -2.43221204e-05 -6.84051420e-07
 -2.44197118e-05]
Mean R-squared Score: -3.2927685734795985e-05


Linear Regression:

In [42]:
from sklearn.linear_model import LinearRegression

scaler = MinMaxScaler()
#scaler = StandardScaler()

lin_reg = LinearRegression()

lr_pipeline = Pipeline([('scaler', scaler),('regressor', lin_reg)])
lr_pipeline.fit(X_train, y_train)

y_pred2 = lr_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred2)
mse = mean_squared_error(y_val, y_pred2)
rmse = mean_squared_error(y_val, y_pred2, squared=False)
r2 = r2_score(y_val, y_pred2)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")

Mean Absolute Error: 7.34
Mean Squared Error: 151.35
Root Mean Squared Error: 12.30
R-squared (R2) Score: 0.77


In [43]:
r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(lr_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())

Cross-Validation R-squared Scores: [0.75127652 0.78092893 0.7878777  0.76792489 0.77780879]
Mean R-squared Score: 0.7731633660158799


Polynomial Regression:

In [44]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression

#scaler = MinMaxScaler()
scaler = StandardScaler()

poly_features = PolynomialFeatures(degree=2)
lin_reg = LinearRegression()

poly_pipeline = Pipeline([('scaler', scaler),('poly', poly_features),('regressor',lin_reg)])
poly_pipeline.fit(X_train, y_train)
y_pred3 = poly_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred3)
mse = mean_squared_error(y_val, y_pred3)
rmse = mean_squared_error(y_val, y_pred3, squared=False)
r2 = r2_score(y_val, y_pred3)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")

Mean Absolute Error: 6.06
Mean Squared Error: 139056.73
Root Mean Squared Error: 372.90
R-squared (R2) Score: -210.72


In [45]:
r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(poly_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())

Cross-Validation R-squared Scores: [ 8.74048660e-01 -1.54316079e+06 -1.14872848e+04 -3.93767510e+04
  9.17042633e-01]
Mean R-squared Score: -318804.6068270002


Polynomial with Ridge Regression:

In [46]:
'''from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge

#scaler = MinMaxScaler()
scaler = StandardScaler()

poly_features = PolynomialFeatures(degree=2)
ridge = Ridge(alpha=1.0)

ridge_poly_pipeline = Pipeline([('scaler', scaler),('poly', poly_features),('regressor',ridge)])
ridge_poly_pipeline.fit(X_train, y_train)
y_pred4 = ridge_poly_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred4)
mse = mean_squared_error(y_val, y_pred4)
rmse = mean_squared_error(y_val, y_pred4, squared=False)
r2 = r2_score(y_val, y_pred4)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")'''

'from sklearn.preprocessing import StandardScaler, PolynomialFeatures\nfrom sklearn.linear_model import Ridge\n\n#scaler = MinMaxScaler()\nscaler = StandardScaler()\n\npoly_features = PolynomialFeatures(degree=2)\nridge = Ridge(alpha=1.0)\n\nridge_poly_pipeline = Pipeline([(\'scaler\', scaler),(\'poly\', poly_features),(\'regressor\',ridge)])\nridge_poly_pipeline.fit(X_train, y_train)\ny_pred4 = ridge_poly_pipeline.predict(X_val)\n\nmae = mean_absolute_error(y_val, y_pred4)\nmse = mean_squared_error(y_val, y_pred4)\nrmse = mean_squared_error(y_val, y_pred4, squared=False)\nr2 = r2_score(y_val, y_pred4)\n\nprint(f"Mean Absolute Error: {mae:.2f}")\nprint(f"Mean Squared Error: {mse:.2f}")\nprint(f"Root Mean Squared Error: {rmse:.2f}")\nprint(f"R-squared (R2) Score: {r2:.2f}")'

In [47]:
'''r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(ridge_poly_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())'''

'r2_scorer = make_scorer(r2_score)\ncv_scores = cross_val_score(ridge_poly_pipeline, X, y, cv=5,scoring = r2_scorer) \n\nprint("Cross-Validation R-squared Scores:", cv_scores)\nprint("Mean R-squared Score:", cv_scores.mean())'

Lasso Regression:

In [48]:
'''from sklearn.linear_model import Lasso

#scaler = MinMaxScaler()
scaler = StandardScaler()

lasso = Lasso(alpha=1.0)

lasso_pipeline = Pipeline([('scaler', scaler),('regressor',lasso)])
lasso_pipeline.fit(X_train, y_train)
y_pred5 = lasso_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred5)
mse = mean_squared_error(y_val, y_pred5)
rmse = mean_squared_error(y_val, y_pred5, squared=False)
r2 = r2_score(y_val, y_pred5)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")'''

'from sklearn.linear_model import Lasso\n\n#scaler = MinMaxScaler()\nscaler = StandardScaler()\n\nlasso = Lasso(alpha=1.0)\n\nlasso_pipeline = Pipeline([(\'scaler\', scaler),(\'regressor\',lasso)])\nlasso_pipeline.fit(X_train, y_train)\ny_pred5 = lasso_pipeline.predict(X_val)\n\nmae = mean_absolute_error(y_val, y_pred5)\nmse = mean_squared_error(y_val, y_pred5)\nrmse = mean_squared_error(y_val, y_pred5, squared=False)\nr2 = r2_score(y_val, y_pred5)\n\nprint(f"Mean Absolute Error: {mae:.2f}")\nprint(f"Mean Squared Error: {mse:.2f}")\nprint(f"Root Mean Squared Error: {rmse:.2f}")\nprint(f"R-squared (R2) Score: {r2:.2f}")'

In [49]:
'''r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(lasso_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())'''

'r2_scorer = make_scorer(r2_score)\ncv_scores = cross_val_score(lasso_pipeline, X, y, cv=5,scoring = r2_scorer) \n\nprint("Cross-Validation R-squared Scores:", cv_scores)\nprint("Mean R-squared Score:", cv_scores.mean())'

Ridge Regression

In [50]:
'''from sklearn.linear_model import Ridge

#scaler = MinMaxScaler()
scaler = StandardScaler()

ridge = Ridge(alpha=1.0)

ridge_pipeline = Pipeline([('scaler', StandardScaler()),('regressor',ridge)])
ridge_pipeline.fit(X_train, y_train)
y_pred6 = ridge_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred6)
mse = mean_squared_error(y_val, y_pred6)
rmse = mean_squared_error(y_val, y_pred6, squared=False)
r2 = r2_score(y_val, y_pred6)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")'''

'from sklearn.linear_model import Ridge\n\n#scaler = MinMaxScaler()\nscaler = StandardScaler()\n\nridge = Ridge(alpha=1.0)\n\nridge_pipeline = Pipeline([(\'scaler\', StandardScaler()),(\'regressor\',ridge)])\nridge_pipeline.fit(X_train, y_train)\ny_pred6 = ridge_pipeline.predict(X_val)\n\nmae = mean_absolute_error(y_val, y_pred6)\nmse = mean_squared_error(y_val, y_pred6)\nrmse = mean_squared_error(y_val, y_pred6, squared=False)\nr2 = r2_score(y_val, y_pred6)\n\nprint(f"Mean Absolute Error: {mae:.2f}")\nprint(f"Mean Squared Error: {mse:.2f}")\nprint(f"Root Mean Squared Error: {rmse:.2f}")\nprint(f"R-squared (R2) Score: {r2:.2f}")'

In [51]:
'''r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(ridge_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())'''

'r2_scorer = make_scorer(r2_score)\ncv_scores = cross_val_score(ridge_pipeline, X, y, cv=5,scoring = r2_scorer) \n\nprint("Cross-Validation R-squared Scores:", cv_scores)\nprint("Mean R-squared Score:", cv_scores.mean())'

KNN-Regressor:

In [52]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

#scaler = MinMaxScaler()
scaler = StandardScaler()

knn = KNeighborsRegressor()

knn_pipeline = Pipeline([('scaler', StandardScaler()),('regressor',knn)])

# Get the best n_neighbors value from the GridSearchCV results:
param_grid = {'regressor__n_neighbors': [7, 9, 11, 13, 15, 17]}
grid_search = GridSearchCV(knn_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_n = grid_search.best_params_['regressor__n_neighbors']

knn_pipeline.set_params(regressor__n_neighbors=best_n)
knn_pipeline.fit(X_train, y_train)
y_pred7 = knn_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred7)
mse = mean_squared_error(y_val, y_pred7)
rmse = mean_squared_error(y_val, y_pred7, squared=False)
r2 = r2_score(y_val, y_pred7)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")

Mean Absolute Error: 6.36
Mean Squared Error: 127.36
Root Mean Squared Error: 11.29
R-squared (R2) Score: 0.81


In [53]:
print(best_n)

11


In [54]:
r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(knn_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())

Cross-Validation R-squared Scores: [0.79882221 0.81925817 0.82828452 0.81374968 0.81943704]
Mean R-squared Score: 0.8159103239314904


KNN-Polynomial:

In [55]:
'''from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

#scaler = MinMaxScaler()
scaler = StandardScaler()

poly_features = PolynomialFeatures(degree=2)
knn = KNeighborsRegressor()

knn_poly_pipeline = Pipeline([('scaler', StandardScaler()),('poly', poly_features), ('regressor',knn)])

# Get the best n_neighbors value from the GridSearchCV results:
param_grid = {'regressor__n_neighbors': [1, 3, 5, 7, 9, 11, 13]}
grid_search = GridSearchCV(knn_poly_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_n2 = grid_search.best_params_['regressor__n_neighbors']

knn_poly_pipeline.set_params(regressor__n_neighbors=best_n2)
knn_poly_pipeline.fit(X_train, y_train)
y_pred8 = knn_poly_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred8)
mse = mean_squared_error(y_val, y_pred8)
rmse = mean_squared_error(y_val, y_pred8, squared=False)
r2 = r2_score(y_val, y_pred8)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")'''

'from sklearn.preprocessing import StandardScaler, PolynomialFeatures\nfrom sklearn.neighbors import KNeighborsRegressor\nfrom sklearn.model_selection import GridSearchCV\n\n#scaler = MinMaxScaler()\nscaler = StandardScaler()\n\npoly_features = PolynomialFeatures(degree=2)\nknn = KNeighborsRegressor()\n\nknn_poly_pipeline = Pipeline([(\'scaler\', StandardScaler()),(\'poly\', poly_features), (\'regressor\',knn)])\n\n# Get the best n_neighbors value from the GridSearchCV results:\nparam_grid = {\'regressor__n_neighbors\': [1, 3, 5, 7, 9, 11, 13]}\ngrid_search = GridSearchCV(knn_poly_pipeline, param_grid, cv=5, scoring=\'neg_mean_squared_error\')\ngrid_search.fit(X_train, y_train)\nbest_n2 = grid_search.best_params_[\'regressor__n_neighbors\']\n\nknn_poly_pipeline.set_params(regressor__n_neighbors=best_n2)\nknn_poly_pipeline.fit(X_train, y_train)\ny_pred8 = knn_poly_pipeline.predict(X_val)\n\nmae = mean_absolute_error(y_val, y_pred8)\nmse = mean_squared_error(y_val, y_pred8)\nrmse = mean_

In [56]:
'''r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(knn_poly_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())'''

'r2_scorer = make_scorer(r2_score)\ncv_scores = cross_val_score(knn_poly_pipeline, X, y, cv=5,scoring = r2_scorer) \n\nprint("Cross-Validation R-squared Scores:", cv_scores)\nprint("Mean R-squared Score:", cv_scores.mean())'

Support Vector Machine (SVM) regressor:

In [57]:
'''from sklearn.svm import SVR

#scaler = MinMaxScaler()
scaler = StandardScaler()

svm = SVR()

svm_pipeline = Pipeline([('scaler', StandardScaler()),('regressor',svm)])
svm_pipeline.fit(X_train, y_train)
y_pred9 = svm_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred9)
mse = mean_squared_error(y_val, y_pred9)
rmse = mean_squared_error(y_val, y_pred9, squared=False)
r2 = r2_score(y_val, y_pred9)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")'''

'from sklearn.svm import SVR\n\n#scaler = MinMaxScaler()\nscaler = StandardScaler()\n\nsvm = SVR()\n\nsvm_pipeline = Pipeline([(\'scaler\', StandardScaler()),(\'regressor\',svm)])\nsvm_pipeline.fit(X_train, y_train)\ny_pred9 = svm_pipeline.predict(X_val)\n\nmae = mean_absolute_error(y_val, y_pred9)\nmse = mean_squared_error(y_val, y_pred9)\nrmse = mean_squared_error(y_val, y_pred9, squared=False)\nr2 = r2_score(y_val, y_pred9)\n\nprint(f"Mean Absolute Error: {mae:.2f}")\nprint(f"Mean Squared Error: {mse:.2f}")\nprint(f"Root Mean Squared Error: {rmse:.2f}")\nprint(f"R-squared (R2) Score: {r2:.2f}")'

In [58]:
'''r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(svm_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())'''

'r2_scorer = make_scorer(r2_score)\ncv_scores = cross_val_score(svm_pipeline, X, y, cv=5,scoring = r2_scorer) \n\nprint("Cross-Validation R-squared Scores:", cv_scores)\nprint("Mean R-squared Score:", cv_scores.mean())'

Decision Tree Regressor:

In [59]:
from sklearn.tree import DecisionTreeRegressor

#scaler = MinMaxScaler()
scaler = StandardScaler()

dtr = DecisionTreeRegressor()

dtr_pipeline = Pipeline([('scaler', StandardScaler()),('regressor',dtr)])
dtr_pipeline.fit(X_train, y_train)
y_pred10 = dtr_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred10)
mse = mean_squared_error(y_val, y_pred10)
rmse = mean_squared_error(y_val, y_pred10, squared=False)
r2 = r2_score(y_val, y_pred10)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")

Mean Absolute Error: 3.88
Mean Squared Error: 73.17
Root Mean Squared Error: 8.55
R-squared (R2) Score: 0.89


In [60]:
r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(dtr_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())

Cross-Validation R-squared Scores: [0.87445411 0.90480757 0.91327836 0.8901924  0.89368488]
Mean R-squared Score: 0.8952834661696674


Bagging and Boosting:

In [61]:
'''from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

#scaler = MinMaxScaler()
scaler = StandardScaler()

ensemble = BaggingRegressor(estimator=AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=3)))

ensemble_pipeline = Pipeline([('scaler', StandardScaler()),('regressor',ensemble)])
ensemble_pipeline.fit(X_train, y_train)
y_pred11 = ensemble_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred11)
mse = mean_squared_error(y_val, y_pred11)
rmse = mean_squared_error(y_val, y_pred11, squared=False)
r2 = r2_score(y_val, y_pred11)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")'''

'from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor\nfrom sklearn.tree import DecisionTreeRegressor\n\n#scaler = MinMaxScaler()\nscaler = StandardScaler()\n\nensemble = BaggingRegressor(estimator=AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=3)))\n\nensemble_pipeline = Pipeline([(\'scaler\', StandardScaler()),(\'regressor\',ensemble)])\nensemble_pipeline.fit(X_train, y_train)\ny_pred11 = ensemble_pipeline.predict(X_val)\n\nmae = mean_absolute_error(y_val, y_pred11)\nmse = mean_squared_error(y_val, y_pred11)\nrmse = mean_squared_error(y_val, y_pred11, squared=False)\nr2 = r2_score(y_val, y_pred11)\n\nprint(f"Mean Absolute Error: {mae:.2f}")\nprint(f"Mean Squared Error: {mse:.2f}")\nprint(f"Root Mean Squared Error: {rmse:.2f}")\nprint(f"R-squared (R2) Score: {r2:.2f}")'

In [62]:
'''r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(ensemble_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())'''

'r2_scorer = make_scorer(r2_score)\ncv_scores = cross_val_score(ensemble_pipeline, X, y, cv=5,scoring = r2_scorer) \n\nprint("Cross-Validation R-squared Scores:", cv_scores)\nprint("Mean R-squared Score:", cv_scores.mean())'

Random Forest Regressor:

In [63]:
from sklearn.ensemble import RandomForestRegressor

#scaler = MinMaxScaler()
scaler = StandardScaler()

rf = RandomForestRegressor(n_estimators=100, random_state=42)

rf_pipeline = Pipeline([('scaler', scaler),('regressor',rf)])
rf_pipeline.fit(X_train, y_train)
y_pred12 = rf_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred12)
mse = mean_squared_error(y_val, y_pred12)
rmse = mean_squared_error(y_val, y_pred12, squared=False)
r2 = r2_score(y_val, y_pred12)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")

Mean Absolute Error: 2.81
Mean Squared Error: 43.50
Root Mean Squared Error: 6.60
R-squared (R2) Score: 0.93


In [64]:
r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(rf_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())

Cross-Validation R-squared Scores: [0.90838933 0.94674761 0.95529696 0.94107558 0.94736888]
Mean R-squared Score: 0.9397756741126182


Gradient Boosting:

In [65]:
from sklearn.ensemble import GradientBoostingRegressor

#scaler = MinMaxScaler()
scaler = StandardScaler()

gbr = GradientBoostingRegressor(n_estimators=100, random_state=42)

gbr_pipeline = Pipeline([('scaler', scaler),('regressor',gbr)])
gbr_pipeline.fit(X_train, y_train)
y_pred13 = gbr_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred13)
mse = mean_squared_error(y_val, y_pred13)
rmse = mean_squared_error(y_val, y_pred13, squared=False)
r2 = r2_score(y_val, y_pred13)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")

Mean Absolute Error: 3.07
Mean Squared Error: 45.02
Root Mean Squared Error: 6.71
R-squared (R2) Score: 0.93


In [66]:
r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(gbr_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())

Cross-Validation R-squared Scores: [0.8912748  0.94389117 0.9481006  0.93038157 0.94170127]
Mean R-squared Score: 0.9310698822614807


Stacking Regressor (KNN, Random Forest, GradientBoosting) 

In [67]:
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

#scaler = MinMaxScaler()
scaler = StandardScaler()

gbr = GradientBoostingRegressor(n_estimators=100, random_state=42)
knn = KNeighborsRegressor(n_neighbors=best_n)
rf = RandomForestRegressor(n_estimators=100, random_state=42)

base_regressors = [('random_forest', rf), ('gradient_boosting', gbr),
                   ('k_nearest_neighbours', knn)]

stack = StackingRegressor(estimators=base_regressors, final_estimator = LinearRegression())

stack_pipeline = Pipeline([('scaler', scaler),('stacking_regressor', stack)])
stack_pipeline.fit(X_train, y_train)
y_pred13 = stack_pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred13)
mse = mean_squared_error(y_val, y_pred13)
rmse = mean_squared_error(y_val, y_pred13, squared=False)
r2 = r2_score(y_val, y_pred13)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")

Mean Absolute Error: 2.80
Mean Squared Error: 40.75
Root Mean Squared Error: 6.38
R-squared (R2) Score: 0.94


In [68]:
r2_scorer = make_scorer(r2_score)
cv_scores = cross_val_score(stack_pipeline, X, y, cv=5,scoring = r2_scorer) 

print("Cross-Validation R-squared Scores:", cv_scores)
print("Mean R-squared Score:", cv_scores.mean())

Cross-Validation R-squared Scores: [0.90724857 0.9505196  0.95717095 0.94200717 0.95007284]
Mean R-squared Score: 0.9414038260890571


Submission:

In [69]:
#test_df=pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv')
#cols = ['passenger_count', 'trip_distance', 'RatecodeID', 'extra', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'Airport_fee']
#imputer = SimpleImputer(strategy='mean')
#test_df[cols] = imputer.fit_transform(test_df[cols])

#X_Test = test_df.drop(columns=['payment_type','store_and_fwd_flag','tpep_pickup_datetime', 'tpep_dropoff_datetime'])
#y_pred3 = model.predict(X_Test)

In [70]:
final_pred = stack_pipeline.predict(test)

In [71]:
test.head()

,passenger_count,trip_distance,PULocationID,DOLocationID,extra,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,...,VendorID_2,RatecodeID_1.0,RatecodeID_1.51830715769907,RatecodeID_2.0,RatecodeID_3.0,RatecodeID_4.0,RatecodeID_5.0,RatecodeID_99.0,RatecodeID_1.5670143713319924,RatecodeID_6.0
0,1.0,4.95,20,3,1.0,6.067612,0.0,1.0,2.5,0.0,...,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2.10,9,81,2.5,6.191269,0.0,1.0,2.5,0.0,...,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.95,92,90,2.5,3.983872,0.0,1.0,2.5,0.0,...,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.80,19,102,3.5,6.839341,0.0,1.0,2.5,0.0,...,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,4.01,131,229,0.0,1.468943,0.0,1.0,0.0,0.0,...,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
X.head()

,passenger_count,trip_distance,PULocationID,DOLocationID,extra,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,...,VendorID_2,RatecodeID_1.0,RatecodeID_1.51830715769907,RatecodeID_2.0,RatecodeID_3.0,RatecodeID_4.0,RatecodeID_5.0,RatecodeID_99.0,RatecodeID_1.5670143713319924,RatecodeID_6.0
0,1.0,2.14,120,9,2.5,7.165589,0.0,1.0,2.5,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,1.0,2.70,15,215,3.5,6.067401,0.0,1.0,2.5,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,1.0,1.15,167,223,0.0,4.111547,0.0,1.0,2.5,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,1.0,0.40,128,239,2.5,6.411079,0.0,1.0,2.5,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,3.0,1.10,203,52,1.0,4.769377,0.0,1.0,2.5,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [73]:
submission = pd.DataFrame(columns = ["ID","total_amount"])
submission["ID"] = [i for i in range(1,len(final_pred)+1)]
submission["total_amount"] = pd.DataFrame(final_pred)
submission.to_csv('submission.csv',index=False)

In [74]:
#submission = pd.DataFrame(columns = ["ID","total_amount"])
#submission["ID"] = [i for i in range(1,len(final_pred)+1)]
#submission["total_amount"] = final_pred
#submission.to_csv('submission.csv',index=False)